# Aim
Randomly sample points (no optimizer) for 00 ->10

In [ ]:
import sys 
sys.path.append('C:\\Users\\Eesh Gupta\\Documents\\RU Research\\Chakram')
sys.path.append('C:\\Users\\Eesh Gupta\\Documents\\RU Research\\Chakram\\Double-ECD\\May\\DECD')

import DECD

In [ ]:
from DECD import BatchOptimizer

In [ ]:
import numpy as np
import tensorflow as tf

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)  # supress warnings
import h5py

# print(
#     "\nNeed tf version 2.3.0 or later. Using tensorflow version: "
#     + tf.__version__
#     + "\n"
# )
import ECD_control.ECD_optimization.tf_quantum as tfq
from ECD_control.ECD_optimization.visualization import VisualizationMixin
import qutip as qt
import datetime
import time


# Code

In [ ]:
#The target oscillator state.
N1 =10
N2 =10
Fock1 = 0
Fock2= 0
psi_i1 = qt.basis(N1,Fock1) #target state
psi_i2 = qt.basis(N2,Fock2)
psi_initial = qt.tensor(psi_i1, psi_i2)

In [ ]:
psi_initial

In [ ]:
#The target oscillator state.
N1 =10
N2 =10
Fock1 = 1
Fock2= 0
psi_t1 = qt.basis(N1,Fock1) #target state
psi_t2 = qt.basis(N2,Fock2)
psi_target = qt.tensor(psi_t1, psi_t2)
psi_target

In [ ]:
#Optimization of ECD Circuit parameters (betas, phis, and thetas)
#the optimization options
opt_params = {
'N_blocks' : 5, #circuit depth
'N_multistart' : 2, #Batch size (number of circuit optimizations to run in parallel)
'epochs' : 1, #number of epochs before termination
'epoch_size' : 20, #number of adam steps per epoch
'learning_rate' : 0.01, #adam learning rate
'term_fid' : 0.995, #terminal fidelitiy
'dfid_stop' : 1e-6, #stop if dfid between two epochs is smaller than this number
'beta_scale' : 3.0, #maximum |beta| for random initialization
'gamma_scale' : 3.0, #maximum |gamma| for random initialization
'N_cav1': N1, #number of levels in mode 1
'N_cav2': N2, #number of levels in mode 2
'initial_states' : [qt.tensor(qt.basis(2,0),psi_initial)], #qubit tensor oscillator, start in |g> |0>
'target_states' : [qt.tensor(qt.basis(2,0), psi_target)], #end in |e> |target>.
'name' : 'Fock1 %d' % Fock1, #name for printing and saving
'filename' : None, #if no filename specified, results will be saved in this folder under 'name.h5'
}


#note: optimizer includes pi pulse in every ECD step. However, final ECD step is implemented 
#in experiment as a displacement since the qubit and oscillator should be disentangled at this point.
#So, we ask the optimizer to end in |e> |target> instead of |g>|target>.

In [ ]:
tf.config.run_functions_eagerly(True)

In [ ]:
#create optimization object. 
#initial params will be randomized upon creation
opt = BatchOptimizer(**opt_params)

#print optimization info. 
opt.print_info()

# Randomly sampling

### First we create a blackbox which given a set of ECD parameters, finds the fidelity

In [ ]:
def blackbox(param_array):
    '''
    Input: array of params
    Output: fidelity
    '''
    
    #since 2 multistarts is minimum in batch optimizer, we create 2 copies of everything
    params_array_double = double_array(param_array)
    #betas_rho, betas_angle, gammas_rho, gammas_angle, alphas1_rho, alphas1_angle, alphas2_rho, alphas2_angle, phis, etas, thetas = params_array_double
    
    #convert from numpy array to tensor
    params_tensor = convert_to_tensor(params_array_double)
    betas_rho, betas_angle, gammas_rho, gammas_angle, alphas1_rho, alphas1_angle, alphas2_rho, alphas2_angle, phis, etas, thetas = params_tensor
    
    #compute fidelity
    fids_tensor =opt.batch_fidelities(
        betas_rho,
        betas_angle,
        gammas_rho,
        gammas_angle,
        alphas1_rho,
        alphas1_angle,
        alphas2_rho,
        alphas2_angle,
        phis,
        etas,
        thetas
        ) 
    fid = fids_tensor[0]
    return float(fid.numpy())

def double_array(arry): 
    '''
    Reason: Batch optimzier cannot work with 1 multistart, so gotta work with 2
    
    Input: given an array [[a,b], [c]]
    Output: [[[a,a], [b,b]], [[c,c]]]
    '''
    new_arry = []
    for arr in arry:
        new_arr = []
        for num in arr: 
            new_arr.append([num,num])
        new_arry.append(new_arr)
    return new_arry

def convert_to_tensor(args):
    '''
    Input: list of numpy arrays
    Output: tensor objects
    '''
    tensors =[]
    for arg in args:
        t = tf.convert_to_tensor(np.array(arg), dtype = tf.float32)
        #print(t.shape)
        tensors.append(t)
    return tensors

In [ ]:
#test
import random
import math
#randomly sample parameters
def random_radius():
    return random.uniform(-3, 3)
def random_angle():
    return random.uniform(-math.pi, math.pi)

def generate_random_params(n_layers):
    '''
    Output: array of subarrays where each subarray is some parameter
    '''
    betas_rho =[ random_radius() for i in range(n_layers)]
    betas_angle = [random_angle() for i in range(n_layers)]
    gammas_rho = [random_radius() for i in range(n_layers)]
    gammas_angle = [random_angle() for i in range(n_layers)]
    alphas1_rho = [0.0]
    alphas1_angle = [0.0]
    alphas2_rho = [0.0]
    alphas2_angle = [0.0]
    phis = [random_angle() for i in range(n_layers)]
    etas= [math.pi/2 for i in range(n_layers)]
    thetas = [random_angle() for i in range(n_layers)]
    
    params = [betas_rho,
        betas_angle,
        gammas_rho,
        gammas_angle,
        alphas1_rho,
        alphas1_angle,
        alphas2_rho,
        alphas2_angle,
        phis,
        etas,
        thetas]
    return params

In [ ]:
blackbox(generate_random_params(5))

In [ ]:
import matplotlib.pyplot as plt
fids = []
params = []
iters = []

for i in range(1000):
    param = generate_random_params(5)
    fid = blackbox(param)
    params.append(param)
    fids.append(fid)
    iters.append(i)
    plt.plot(iters, fids,marker = 'o' )
    plt.show()

In [ ]:
max(fids)